# Strategy Serialization with BoFire

## Imports

In [1]:
import json
from pydantic import parse_obj_as


from bofire.data_models.domain.api import Inputs, Outputs, Domain
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.multi import CrossCoupling
from bofire.data_models.strategies.api import SoboStrategy as SoboStrategyDataModel
from bofire.data_models.strategies.api import QnehviStrategy as QnehviStrategyDataModel
from bofire.data_models.strategies.api import RandomStrategy as RandomStrategyDataModel
from bofire.data_models.strategies.api import AnyStrategy
from bofire.data_models.acquisition_functions.api import qNEI
import bofire.strategies.api as stategies




/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Single Objective Problem Setup

In [2]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=10)
experiments = benchmark.f(samples, return_complete=True)

## Random Strategy

In [3]:
# setup the data model
domain = Domain(input_features=benchmark.domain.input_features)
strategy_data = RandomStrategyDataModel(domain=domain)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "RandomStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "output_features": {"type": "Outputs", "features": []}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 279}'

In [4]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
stategy = stategies.map(strategy_data)

# ask it
df_candidates = stategy.ask(candidate_count=5)

# transform to spec
candidates = stategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=-3.744742482549616), 'x_2': InputValue(value=3.4578032920821746)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-3.152299000916403), 'x_2': InputValue(value=2.188805709088892)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=2.378973293392475), 'x_2': InputValue(value=1.5645555272201932)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-1.138877235330762), 'x_2': InputValue(value=-2.345015446113318)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-0.4336374168749888), 'x_2': InputValue(value=-0.4229157761068656)}, outputValues=None)]

## SOBO Strategy

This will fail as SOBO is a predictive strategy which needs also output feature definitions, which is missing in the domain from before.

In [24]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

ValidationError: 1 validation error for SoboStrategy
domain
  no output feature specified (type=value_error)

Next try with a correct domain:

In [3]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=benchmark.domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

[] ['y']
[SingleTaskGPSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInput(type='ContinuousInput', key='x_2', lower_bound=-4.0, upper_bound=4.0)]), output_features=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', objective=MaximizeObjective(type='MaximizeObjective', w=1.0, lower_bound=0, upper_bound=1))]), input_preprocessing_specs={}, kernel=ScaleKernel(type='ScaleKernel', base_kernel=MaternKernel(type='MaternKernel', ard=True, nu=2.5, lengthscale_prior=GammaPrior(type='GammaPrior', concentration=3.0, rate=6.0)), outputscale_prior=GammaPrior(type='GammaPrior', concentration=2.0, rate=0.15)), scaler=<ScalerEnum.NORMALIZE: 'NORMALIZE'>)]
surrogates=[SingleTaskGPSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, up

'{"type": "SoboStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "output_features": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MaximizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 100, "num_sobol_samples": 512, "num_restarts": 8, "num_raw_samples": 1024, "descriptor_method": "EXHAUSTIVE", "categorical_method": "EXHAUSTIVE", "discrete_method": "EXHAUSTIVE", "surrogate_specs": {"surrogates": [{"type": "SingleTaskGPSurrogate", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "lower_bound": -4.0, "upper_bound": 4.0}, {"type": "ContinuousInput", "key": "x_2", "lower_bound": -4.0, "upper_bound": 4.0}]}, "ou

In [4]:
strategy_data.surrogate_specs

BotorchSurrogates(surrogates=[SingleTaskGPSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInput(type='ContinuousInput', key='x_2', lower_bound=-4.0, upper_bound=4.0)]), output_features=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', objective=MaximizeObjective(type='MaximizeObjective', w=1.0, lower_bound=0, upper_bound=1))]), input_preprocessing_specs={}, kernel=ScaleKernel(type='ScaleKernel', base_kernel=MaternKernel(type='MaternKernel', ard=True, nu=2.5, lengthscale_prior=GammaPrior(type='GammaPrior', concentration=3.0, rate=6.0)), outputscale_prior=GammaPrior(type='GammaPrior', concentration=2.0, rate=0.15)), scaler=<ScalerEnum.NORMALIZE: 'NORMALIZE'>)])

In [5]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
#stategy = stategies.map(strategy_data)

# ask it
#df_candidates = stategy.ask(candidate_count=5)

# transform to spec
#candidates = stategy.to_candidates(df_candidates)

#candidates

['y'] []
[BotorchSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInput(type='ContinuousInput', key='x_2', lower_bound=-4.0, upper_bound=4.0)]), output_features=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', objective=MaximizeObjective(type='MaximizeObjective', w=1.0, lower_bound=0.0, upper_bound=1.0))]), input_preprocessing_specs={})]
surrogates=[BotorchSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInput(type='ContinuousInput', key='x_2', lower_bound=-4.0, upper_bound=4.0)]), output_features=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', objective=MaximizeObjective(type='MaximizeObjective', w=1.0, lower_bound=0.0, upper_bound=1.0))]), input_p

In [9]:
strategy_data.surrogate_specs

BotorchSurrogates(surrogates=[BotorchSurrogate(type='SingleTaskGPSurrogate', input_features=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInput(type='ContinuousInput', key='x_2', lower_bound=-4.0, upper_bound=4.0)]), output_features=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', objective=MaximizeObjective(type='MaximizeObjective', w=1.0, lower_bound=0.0, upper_bound=1.0))]), input_preprocessing_specs={})])